In [3]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,SimpleRNN,Dense


In [4]:
## Load the word index
word_index=imdb.get_word_index()
reverse_word_index={value:key for (key,value) in word_index.items()}

In [5]:
## Load the pre-trained model with relu activation
model = tf.keras.models.load_model('imdb_rnn_model.h5')
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ (32, 500, 128)         │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_2 (SimpleRNN)        │ (32, 500, 128)         │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_3 (SimpleRNN)        │ (32, 128)              │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (32, 1)                │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,345,923 (5.13 MB)

 Trainable params: 1,345,921 (5.13 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

In [6]:
model.get_weights()

[array([[ 2.95788236e-02, -4.71554324e-02, -8.69909376e-02, ...,
          5.97934946e-02,  7.41769373e-03, -1.99614167e-02],
        [ 4.57883440e-02, -3.85574549e-02,  1.36167239e-02, ...,
          2.22623479e-02,  3.09061399e-03, -5.40142283e-02],
        [ 1.15171336e-02, -6.11413307e-02,  7.50408396e-02, ...,
          6.30784705e-02, -6.40656948e-02, -5.81673719e-03],
        ...,
        [-1.24790251e-01, -5.74413985e-02, -1.07127205e-01, ...,
         -8.75974447e-02, -1.47094848e-02,  1.10614911e-01],
        [-1.39368340e-01, -5.56678474e-02, -8.82665664e-02, ...,
         -1.53284166e-02,  5.40561490e-02,  1.24326255e-02],
        [-1.04538381e-01, -6.15835674e-02, -1.39747396e-01, ...,
          2.93358527e-02, -1.58223600e-04,  2.01958567e-01]],
       shape=(10000, 128), dtype=float32),
 array([[-0.2005448 , -0.29310888,  0.12946662, ...,  0.32363024,
         -0.10411037, -0.21576409],
        [-0.11850333, -0.07781281, -0.15020217, ...,  0.08022376,
         -0.0341564

In [7]:
## Step 2 : helper functions
## Function to decode review
def decode_review(encode_review):
    return ' '.join([reverse_word_index.get(i-3,'?') for i in encode_review])

## Function to Preprocess the user input
def preprocess_text(text):
    # Tokenize the text
    tokens = text.lower().split()
    # Convert tokens to integers based on the word index
    encoded = [word_index.get(word,2)+3 for word in tokens] # 2 is for unknown words
    # Pad the sequence to ensure it matches the input length of the model
    padded = sequence.pad_sequences([encoded],maxlen=256)
    return padded

In [8]:
## Prediction function
def predict_sentiment(text):
    # Preprocess the input text
    processed_text = preprocess_text(text)
    # Make prediction
    prediction = model.predict(processed_text)
    # Interpret the prediction
    sentiment = 'Positive' if prediction[0][0] > 0.5 else 'Negative'
    return sentiment, prediction[0][0]

In [10]:
## Step 4 : User input and prediction
example_review = "The movie was fantastic! I really loved it and would watch it again."
sentiment, confidence = predict_sentiment(example_review)
print(f"Review: {example_review}\nPredicted Sentiment: {sentiment} (Confidence: {confidence:.2f})")
print(f'Prediction Score: {confidence:.3f}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 473ms/step
Review: The movie was fantastic! I really loved it and would watch it again.
Predicted Sentiment: Positive (Confidence: 0.65)
Prediction Score: 0.649
